# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import export_image
from utils import map_image

In [57]:
# region of interest is Amazonia
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/raw/biomes_ibge").first().geometry()
# For specific biomes, the feature "id" is:
    # 18413 = Amazon
    # 18414 = Caatinga
    # 18415 = Cerrado
    # 18416 = Mata Atlantica
    # 18417 = Pampa
    # 18418 = Pantanal

In [106]:
LU_sum = ee.Image('projects/ee-ana-zonia/assets/LU_sum')
fallow = ee.Image('projects/ee-ana-zonia/assets/fallow')
last_fire = ee.Image('projects/ee-ana-zonia/assets/last_fire')
num_fires = ee.Image('projects/ee-ana-zonia/assets/num_fires')
frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020')
med_mat = ee.Image('projects/ee-ana-zonia/assets/median_mature')
last_LU = ee.Image('projects/ee-ana-zonia/assets/last_LU')

land_use = LU_sum.addBands([last_LU, fallow, last_fire, num_fires, frag, med_mat])

ecoreg = ee.Image("projects/ee-ana-zonia/assets/ecoregions_br")
indig = ee.Image("projects/ee-ana-zonia/assets/indig")
soil = ee.Image("projects/ee-ana-zonia/assets/soil")
protec = ee.Image("projects/ee-ana-zonia/assets/protec")
biomes = ee.Image("projects/ee-ana-zonia/assets/biome_br")
categorical = ecoreg.addBands([indig, soil, protec, biomes])

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si").float()
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec").float()
cwd = ee.Image("projects/ee-ana-zonia/assets/raw/cwd_chave").float()
climate = yearly_si.addBands([mean_prec])

biomass = ee.Image('projects/ee-ana-zonia/assets/biomass_masked')
total_export = categorical.addBands([land_use, climate, biomass]).updateMask(mask)

nat_cover = lulc.select('classification_1985').updateMask((lulc.neq(1).neq(4)).neq(5)).lt(7)
total_export = total_export.updateMask(nat_cover)

NameError: name 'lulc' is not defined

In [61]:
img = total_export
name = "total_export"

# Create the export task
task = ee.batch.Export.image.toAsset(
    image = img,
    description = f'{name}',
    assetId = f'projects/ee-ana-zonia/assets/{name}',
    crs = 'EPSG:4326',
    crsTransform = 
    maxPixels = 4e12,
    region=img.geometry()
)

# Start the export task
task.start()

In [102]:
## stratify sample points per feature
stratified = final.stratifiedSample(
    numPoints = 10000,
    classBand = 'ecoreg',
    geometries = True
)

# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toDrive(
    collection=stratified,
    description='final_w_biomass'
)
task.start()


In [2]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_123032_20140515').select(
    ['B4', 'B3', 'B2']
)

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])


# Retrieve the projection information from a band of the original image.
# Call getInfo() on the projection to request a client-side object containing
# the crs and transform information needed for the client-side Export function.
projection = landsat.select('B2').projection().getInfo()

# Export the image, specifying the CRS, transform, and region.
task = ee.batch.Export.image.toDrive(
    image=landsat,
    description='imageToDriveExample_transform',
    crs=projection['crs'],
    crsTransform=projection['transform'],
    region=geometry,
)
task.start()